# Sequency-ordered (Walsh) Hadamard transform

https://ieeexplore.ieee.org/document/1162377

##  Imports

In [1]:
from __future__ import print_function, division
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import torch.nn.functional as F
import imageio
import cv2
import fht
import matplotlib.pyplot as plt
import scipy.io as sio
import PIL
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sys
import wget
import zipfile
from pathlib import Path

## Import important functions from spyrit

In [2]:
from spyrit.learning.model_Had_DCAN import *# compNet, Stat_had, Weight_Decay_Loss
from spyrit.learning.nets import *
from spyrit.misc.disp import *
from spyrit.misc.metrics import *


## User-defined global parameters 

In [3]:
#- Acquisition
img_size = 64 # image size
batch_size = 1024
M = 333       # number of neasurements

#- Model and data paths
model_root = Path('./models/')
stats_root = Path('./stats_ordered2/')
data_root = Path('./data/')
#- Save plot using type 1 font
plt.rcParams['pdf.fonttype'] = 42

print(model_root)
print(data_root)

models
data


## Load STL-10 dataset

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.manual_seed(7)

transform = transforms.Compose(
    [transforms.functional.to_grayscale,
     transforms.Resize((img_size, img_size)),
     transforms.ToTensor(),
     transforms.Normalize([0.5], [0.5])])


trainset = \
    torchvision.datasets.STL10(root=data_root, split='train+unlabeled',download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,shuffle=True)

testset = \
    torchvision.datasets.STL10(root=data_root, split='test',download=True, transform=transform)
testloader =  torch.utils.data.DataLoader(testset, batch_size=batch_size,shuffle=False)

dataloaders = {'train':trainloader, 'val':testloader}

Files already downloaded and verified
Files already downloaded and verified


## Walsh ordered in 2D

In [15]:
def abs_walsh_ordered(dataloader, H1d, tot_num):
    #-- Accumulate over all images in dataset
    n = 0
    Abs_had = np.zeros((H1d.shape[1],H1d.shape[1]))
    for inputs,_ in dataloader:
        inputs = inputs.cpu().detach().numpy();
        for i in range(inputs.shape[0]):
            img = inputs[i,0,:,:];
            h_img = walsh_ordered2(img,H1d);
            Abs_had += abs(h_img);
            n = n+1
        print(f'Abs:  {n} / (less than) {tot_num} images', end='\r')
    print('', end='\n')
    
    #-- Normalize
    Abs_had = Abs_had/n;
    
    return Abs_had


def stat_walsh_ordered(dataloader, root):
    """ 
        Computes Mean Hadamard Image over the whole dataset + 
        Covariance Matrix Amongst the coefficients
    """
    inputs, classes = next(iter(dataloader))
    inputs = inputs.cpu().detach().numpy();
    (batch_size, channels, nx, ny) = inputs.shape;
    tot_num = len(dataloader)*batch_size;
    
    H1d = wh.walsh_ordered(nx)
    
     # Abs matrix
    Mean_had = abs_walsh_ordered(dataloader, H1d, tot_num)
    print("Saving abs")
    np.save(root / Path('Abs_{}x{}'.format(nx,ny)+'.npy'), Mean_had)

    # Mean matrix
    #-- Accumulate over all images in dataset
    n = 0
    Mean_had = np.zeros((nx, ny));
    for inputs,_ in dataloader:
        inputs = inputs.cpu().detach().numpy();
        for i in range(inputs.shape[0]):
            img = inputs[i,0,:,:];
            h_img = walsh_ordered2(img,H1d);
            Mean_had += h_img;
            n = n+1
        print(f'Mean:  {n} / (less than) {tot_num} images', end='\r')
    print('', end='\n')
    
    #-- Normalize & save
    Mean_had = Mean_had/n;
    print("Saving mean")
    np.save(root / Path('Mean_{}x{}'.format(nx,ny)+'.npy'), Mean_had)
    
    # Covariance matrix    
    n = 0
    Cov_had = np.zeros((nx*ny, nx*ny));
    for inputs,_ in dataloader:
        inputs = inputs.cpu().detach().numpy();
        for i in range(inputs.shape[0]):
            img = inputs[i,0,:,:];
            h_img = walsh_ordered2(img, H1d);
            Norm_Variable = np.reshape(h_img-Mean_had, (nx*ny,1));
            Cov_had += Norm_Variable*np.transpose(Norm_Variable);
            n = n+1
        print(f'Covariance:  {n} / (less than) {tot_num} images', end='\r')     
    print()
    
    #-- Normalize & save
    Cov_had = Cov_had/(n-1);  
    np.save(root / Path('Cov_{}x{}'.format(nx,ny)+'.npy'), Cov_had)

In [16]:
path = Path('./stats_ordered')

#if not path.exists():
#    path.mkdir()
        
stat_walsh_ordered(trainloader, path)


Abs:  105000 / (less than) 105472 images
Saving abs
Mean:  105000 / (less than) 105472 images
Saving mean
Covariance:  105000 / (less than) 105472 images


In [ ]:
print('Loading Cov and Mean') 
Cov_had = np.load(stats_root / Path("Cov_{}x{}.npy".format(img_size, img_size)))

In [ ]:
H1d = wh.walsh_ordered(64)
H1d.shape[1]